In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
current_dir = os.getcwd()
dir_warehouse = f"{current_dir}/warehouse"

In [3]:
spark = SparkSession.builder \
    .appName("IcebergWithSpark") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", dir_warehouse) \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .getOrCreate()

25/01/07 20:38:27 WARN Utils: Your hostname, dell resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp0s20f3)
25/01/07 20:38:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/07 20:38:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# Exclui a tabela se existir
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas_partitioned")

DataFrame[]

In [9]:
# Cria tabela particionada Iceberg
spark.sql("""
CREATE TABLE hadoop_catalog.default.vendas_partitioned (
    id INT,
    produto STRING,
    quantidade INT,
    preco DOUBLE,
    data_venda DATE,
    categoria STRING
)
USING iceberg
PARTITIONED BY (categoria, days(data_venda))
""")

DataFrame[]

In [10]:
# inserimos vendas
data = [
    (1, "Produto A", 10, 15.5, "2023-11-01", "Eletronicos"),
    (2, "Produto B", 5, 22.0, "2023-11-02", "Vestuario"),
    (3, "Produto C", 8, 30.0, "2023-11-03", "Eletronicos"),
    (4, "Produto D", 12, 25.0, "2023-11-04", "Alimentos"),
    (5, "Produto E", 7, 18.5, "2023-11-05", "Vestuario"),
    (6, "Produto F", 9, 20.0, "2023-11-06", "Alimentos"),
    (7, "Produto G", 15, 35.0, "2023-11-07", "Eletronicos"),
    (8, "Produto H", 1, 35.0, "2023-11-07", "Eletronicos")
]
columns = ["id", "produto", "quantidade", "preco", "data_venda", "categoria"]

df = spark.createDataFrame(data, columns)
df = df.withColumn("data_venda", F.to_date(F.col("data_venda"), "yyyy-MM-dd"))

df.writeTo("hadoop_catalog.default.vendas_partitioned").append()

In [12]:
# Listamos as Partições
partitions_df = spark.sql("SELECT * FROM hadoop_catalog.default.vendas_partitioned.partitions")
partitions_df.orderBy("partition").show(truncate=False)

+-------------------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|partition                |spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|last_updated_at        |last_updated_snapshot_id|
+-------------------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|{Alimentos, 2023-11-04}  |0      |1           |1         |1711                         |0                           |0                         |0                           |0                         |2025-01-07 20:52:58.273|272615915861596972    

In [13]:
# Deve excluir registro de número 5, partição  Vestuário, 2023-11-05
spark.sql("""
DELETE FROM hadoop_catalog.default.vendas_partitioned
WHERE categoria = 'Vestuario' AND data_venda = '2023-11-05'
""")

DataFrame[]

In [14]:
# Partições Atualizadas
partitions_df = spark.sql("SELECT * FROM hadoop_catalog.default.vendas_partitioned.partitions")
partitions_df.orderBy("partition").show(truncate=False)

+-------------------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|partition                |spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|last_updated_at        |last_updated_snapshot_id|
+-------------------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|{Alimentos, 2023-11-04}  |0      |1           |1         |1711                         |0                           |0                         |0                           |0                         |2025-01-07 20:52:58.273|272615915861596972    

In [15]:
# Categoria Eletrônicos em 7 / 11
filtered_df = spark.sql("""
SELECT * FROM hadoop_catalog.default.vendas_partitioned
WHERE categoria = 'Eletronicos' AND data_venda = '2023-11-07'
""")
filtered_df.show()

+---+---------+----------+-----+----------+-----------+
| id|  produto|quantidade|preco|data_venda|  categoria|
+---+---------+----------+-----+----------+-----------+
|  7|Produto G|        15| 35.0|2023-11-07|Eletronicos|
|  8|Produto H|         1| 35.0|2023-11-07|Eletronicos|
+---+---------+----------+-----+----------+-----------+



In [16]:
# Vamos criar uma tabela clientes. Primeiro verificamos se existe
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.clientes_partitioned")

# Partição por cidade - categoria preferida deve ser as existentes em vendas
spark.sql("""
CREATE TABLE hadoop_catalog.default.clientes_partitioned (
    cliente_id INT,
    nome STRING,
    cidade STRING,
    categoria_preferida STRING
)
USING iceberg
PARTITIONED BY (cidade)
""")

DataFrame[]

In [17]:
# Dados
customer_data = [
    (1, "Alice", "São Paulo", "Eletronicos"),
    (2, "Bob", "Rio de Janeiro", "Vestuario"),
    (3, "Carol", "São Paulo", "Alimentos"),
    (4, "David", "Porto Alegre", "Eletronicos"),
    (5, "Eve", "Novo Hamburgo", "Vestuario")
]
customer_columns = ["cliente_id", "nome", "cidade", "categoria_preferida"]

df_customers = spark.createDataFrame(customer_data, customer_columns)
df_customers.writeTo("hadoop_catalog.default.clientes_partitioned").append()

In [18]:
# Join vendas e clientes pela categoria
join_df = spark.sql("""
SELECT v.id, v.produto, v.quantidade, v.preco, v.data_venda, v.categoria,
        c.cliente_id, c.nome, c.cidade
FROM hadoop_catalog.default.vendas_partitioned v
INNER JOIN hadoop_catalog.default.clientes_partitioned c
        ON v.categoria = c.categoria_preferida
""")

join_df.show()

+---+---------+----------+-----+----------+-----------+----------+-----+--------------+
| id|  produto|quantidade|preco|data_venda|  categoria|cliente_id| nome|        cidade|
+---+---------+----------+-----+----------+-----------+----------+-----+--------------+
|  7|Produto G|        15| 35.0|2023-11-07|Eletronicos|         4|David|  Porto Alegre|
|  7|Produto G|        15| 35.0|2023-11-07|Eletronicos|         1|Alice|     São Paulo|
|  8|Produto H|         1| 35.0|2023-11-07|Eletronicos|         4|David|  Porto Alegre|
|  8|Produto H|         1| 35.0|2023-11-07|Eletronicos|         1|Alice|     São Paulo|
|  4|Produto D|        12| 25.0|2023-11-04|  Alimentos|         3|Carol|     São Paulo|
|  2|Produto B|         5| 22.0|2023-11-02|  Vestuario|         2|  Bob|Rio de Janeiro|
|  2|Produto B|         5| 22.0|2023-11-02|  Vestuario|         5|  Eve| Novo Hamburgo|
|  1|Produto A|        10| 15.5|2023-11-01|Eletronicos|         4|David|  Porto Alegre|
|  1|Produto A|        10| 15.5|

In [19]:
# Join com filtro baseado em cidade e data
filtered_join_df = spark.sql("""
SELECT 
    v.id, v.produto, v.quantidade, v.preco, v.data_venda, v.categoria,
    c.cliente_id, c.nome, c.cidade
FROM hadoop_catalog.default.vendas_partitioned v
INNER JOIN hadoop_catalog.default.clientes_partitioned c
    ON v.categoria = c.categoria_preferida
WHERE c.cidade = 'São Paulo' AND v.data_venda >= '2023-11-03'
""")

filtered_join_df.show()

+---+---------+----------+-----+----------+-----------+----------+-----+---------+
| id|  produto|quantidade|preco|data_venda|  categoria|cliente_id| nome|   cidade|
+---+---------+----------+-----+----------+-----------+----------+-----+---------+
|  7|Produto G|        15| 35.0|2023-11-07|Eletronicos|         1|Alice|São Paulo|
|  8|Produto H|         1| 35.0|2023-11-07|Eletronicos|         1|Alice|São Paulo|
|  4|Produto D|        12| 25.0|2023-11-04|  Alimentos|         3|Carol|São Paulo|
|  6|Produto F|         9| 20.0|2023-11-06|  Alimentos|         3|Carol|São Paulo|
|  3|Produto C|         8| 30.0|2023-11-03|Eletronicos|         1|Alice|São Paulo|
+---+---------+----------+-----+----------+-----------+----------+-----+---------+

